In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [21]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.exceptions import UnknownLevelWarning
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursiveMultiSeries


# Fixtures
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import series
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import exog
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import exog_predict

transformer_exog = ColumnTransformer(
                       [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

In [22]:
import re
import pytest
import numpy as np
import pandas as pd
from skforecast.direct import ForecasterDirectMultiVariate
from sklearn.linear_model import LinearRegression
from sklearn.exceptions import NotFittedError
from sklearn.preprocessing import StandardScaler


In [31]:
forecaster = ForecasterDirectMultiVariate(LinearRegression(), level='l1',
                                            lags=3, steps=3, 
                                            transformer_series=StandardScaler())
forecaster.fit(series=series)
forecaster.out_sample_residuals_ = {
    1: np.array([1, 2, 3, 4, 5]),
    2: np.array([1, 2, 3, 4, 5])
}

err_msg = re.escape(
    "Not `forecaster.out_sample_residuals_` for steps: {3}. Use method "
    "`set_out_sample_residuals()`."
)
with pytest.raises(ValueError, match = err_msg):
    forecaster.predict_bootstrapping(steps=3, use_in_sample_residuals=False)

In [14]:
results

{1: array([0.23145502, 0.23145502, 0.23145502, 0.23145502, 0.23145502]),
 2: array([0.46291005, 0.46291005, 0.46291005, 0.46291005, 0.46291005])}

In [13]:
expected

{1: array([1, 1, 1, 1, 1]), 2: array([2, 2, 2, 2, 2])}